In [2]:
import pandas as pd
import numpy as np
from glob import glob
import re
import pylab as plt
import seaborn as sns
%matplotlib inline

In [3]:
pub_ds2_folder = '/Users/yhuang10/Documents/TEX/pub_ds2_/urisk/pnas/Figures/data'

In [4]:
for target_type in ['case']:#, 'death']: 
    print(f'\n=========== {target_type} ===========')
    otherTeam_fname = f'../otherTeam/data_processed/otherTeam_{target_type}.csv'
    df_otherTeams = pd.read_csv(otherTeam_fname, index_col=0)
    
    ours_fname = f'results_time/methodCompare_{target_type}_ours.csv'
    df_ours = pd.read_csv(ours_fname, index_col=0)[['gnd', 'prd-0_1']].rename(columns={'prd-0_1': 'UnIT'})
    # print(df_ours)
    
    df_combined = df_ours.join(df_otherTeams, how='outer')# .dropna().
    df_combined = df_combined.loc['2020-07-18':]
    # print(df_combined)
    
    df_err = pd.DataFrame(index=df_combined.index)
    for col in df_combined.columns:
        if col == 'gnd':
            continue
        df_err[col] = df_combined[col] - df_combined['gnd']
    mabs = df_err.abs().mean().sort_values(ascending=False)#[-10:]
    df_mabs = pd.DataFrame({'team': mabs.index, 'err': mabs.values})
    df_mabs.to_csv(f'results_time/compare_with_otherTeams_{target_type}_err_raw.csv', index=False, float_format='%.0f')
    # rmse = df_err.pow(2).mean().pow(.5).sort_values()

    print(mabs)
    # print(rmse)

    df_combined = pd.concat([df_otherTeams, df_ours], axis=1, join='outer').round(0)
    df_combined.index.name='date'
    df_combined.sort_index(inplace=True)
    df_combined.to_csv(f'results_time/compare_with_otherTeams_{target_type}.csv', float_format='%.0f')
    # df_combined.to_csv(f'{pub_ds2_folder}/compare_with_otherTeams_{target_type}.csv', float_format='%.0f')


=========== case ===========
JHU_IDD-CovidSP          229597.307692
CovidAnalytics-DELPHI    198249.259259
RobertWalraven-ESG       124200.769231
CU-scenario_low          113274.130435
CU-scenario_high         111792.913043
CU-select                111602.086957
CU-scenario_mid          111101.347826
CU-nochange              110353.625000
IowaStateLW-STEM         108687.846154
Covid19Sim-Simulator     106336.600000
Columbia_UNC-SurvCon     104005.296296
LANL-GrowthRate           99185.444444
UCLA-SuEIR                97286.592593
UMich-RidgeTfReg          90728.037037
OliverWyman-Navigator     89788.185185
COVIDhub-baseline         89520.740741
LNQ-ens1                  88242.851852
USC-SI_kJalpha            86511.222222
COVIDhub-ensemble         83615.384615
UnIT                      80196.851852
Geneva-DetGrowth          77909.481481
QJHong-Encounter          72494.074074
dtype: float64


In [21]:
err_dfs = [] 
for target_type in ['case', 'death']:
    tmp = pd.read_csv(f'results_time/compare_with_otherTeams_{target_type}_err_raw.csv', index_col=0)[-15:]
    tmp.rename(columns={'err': f'err_{target_type}'}, inplace=True)
    err_dfs.append(tmp)
    
df_err = pd.concat(err_dfs, axis=1,join='outer')
df_err.to_csv(f'results_time/compare_with_otherTeams.csv')
df_err.dropna(inplace=True)

In [22]:
func = lambda x: x.split('-')[0].replace('_', '')


def foo(w):
    if w.isupper() or w[0].isupper():
        return w
    elif w.islower():
        return w.capitalize()
    else:
        return w[0].upper() + w[1:]
bar = lambda x: ''.join([foo(w) for w in re.split('_|-', x)])[:12]
    
df_err_case = df_err[['err_case']].sort_values(by='err_case', ascending=False)[-15:].copy()
df_err_case.rename(index={ind: bar(ind) for ind in df_err_case.index}, inplace=True)
df_err_case.index.name = 'team'
df_err_case.to_csv(f'results_time/compare_with_otherTeams_case_err_raw.csv')
# df_err_case.to_csv(f'{pub_ds2_folder}/compare_with_otherTeams_case_err_raw.csv')

df_err_death = df_err[['err_death']].sort_values(by='err_death', ascending=False)[-15:].copy()
df_err_death.rename(index={ind: bar(ind) for ind in df_err_death.index}, inplace=True)
df_err_death.index.name = 'team'
df_err_death.to_csv(f'results_time/compare_with_otherTeams_death_err_raw.csv')
# df_err_death.to_csv(f'{pub_ds2_folder}/compare_with_otherTeams_death_err_raw.csv')